In [1]:
import pandas as pd
from __future__ import division
import csv
from datetime import datetime
from collections import Counter

In [2]:
import matplotlib.pyplot as plt

import stuff

In [3]:
data = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160402.txt')

read data in + merge data

In [4]:
data.columns = data.columns.str.replace(' ','')

remove spaces from exits column

In [5]:
data["DATETIME"] = data["DATE"] + ' ' + data["TIME"]

In [6]:
data['DATETIME'] = pd.to_datetime(data.DATETIME, format='%m/%d/%Y %H:%M:%S')

create datetime and set format

In [7]:
data['HOUR'] = data.DATETIME.apply(lambda x: x.hour)

In [8]:
data['WEEKDAY']=data.DATETIME.apply(lambda x: 'Weekday' if 0<=x.weekday()<5 else 'Weekend')

#same as above in 2 steps
#weekdaycheck = lambda x: 'Weekday' if 0<=x.weekday()<5 else 'Weekend'
#data['WEEKDAY']=data.DATETIME.apply(weekdaycheck)


In [9]:
daysofweek = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

whatdayofweek = lambda x: daysofweek[x.weekday()]


data['DAYOFWEEK'] = data.DATETIME.apply(whatdayofweek)
#data['DAYOFWEEKVALUE'] = data.DATETIME.apply(dayofweekvalue)

create day of week by applying function to DATETIME

In [10]:
data['dENTRIES'] = data.ENTRIES.shift(-1) - data.ENTRIES
data['dEXITS'] = data.EXITS.shift(-1) - data.EXITS
data['dTOTAL'] = data.dENTRIES+ data.dEXITS

###for considering deltas as time(i+1)-time(i)

create deltas by shifting from one above

In [11]:
#data['dENTRIES'] = data.ENTRIES - data.ENTRIES.shift(1)
#data['dEXITS'] = data.EXITS - data.EXITS.shift(1)
#data['dTOTAL'] = data.dENTRIES+ data.dEXITS

###this is for considering deltas as time(i)-time(i-1)

In [12]:
datatest = data[(data.HOUR ==0)|(data.HOUR ==1)]
# same as mask

filter based on time we care about

In [13]:
datatest2 = datatest[(datatest.dENTRIES >= 0) & (datatest.dENTRIES <=5000) & (datatest.dEXITS >= 0) & (datatest.dEXITS <=5000)]
# mask2 = (datatest.dENTRIES >= 0) & (datatest.dENTRIES <=5000) & (datatest.dEXITS >= 0) & (datatest.dEXITS <=5000)


filter a second time based on change in deltas to account for switching machine/station etc.  
Lose out on 29835 - 29522 = 313 total entries  
313 is 1.05% of our data that we lose

In [14]:
final_weekday_or_end = datatest2.groupby([datatest2.WEEKDAY, datatest2.STATION]).sum()

In [15]:
weekend_or_day_final =  (final_weekday_or_end
    .reset_index()
    .sort_values(["WEEKDAY", "dTOTAL"], ascending=[True, False])
)

In [16]:
weekend_or_day_final = weekend_or_day_final.drop(['ENTRIES', 'EXITS', 'HOUR', 'dENTRIES', 'dEXITS'],1)

In [17]:
for r in range(weekend_or_day_final.shape[0]):
    if weekend_or_day_final.ix[r,'WEEKDAY'] =='Weekday':
        weekend_or_day_final.ix[r,'dTOTALACTUAL'] = weekend_or_day_final.ix[r, 'dTOTAL']/5
    else:
        weekend_or_day_final.ix[r,'dTOTALACTUAL'] = weekend_or_day_final.ix[r, 'dTOTAL']/2

In [18]:
weekend_or_day_final = weekend_or_day_final.drop(['dTOTAL'],1)


In [19]:
weekend_or_day_final.head()

,WEEKDAY,STATION,dTOTALACTUAL
345,Weekday,TIMES SQ-42 ST,3975.0
59,Weekday,34 ST-HERALD SQ,2582.4
251,Weekday,JKSN HT-ROOSVLT,2492.8
61,Weekday,34 ST-PENN STA,2447.4
9,Weekday,125 ST,2416.4
227,Weekday,GRD CNTRL-42 ST,2164.0
111,Weekday,96 ST,2130.6
14,Weekday,14 ST-UNION SQ,1863.6
46,Weekday,23 ST,1818.6
84,Weekday,59 ST COLUMBUS,1759.4


weekend_or_day_final is the FINAL dataframe for weekday vs weekend

In [21]:
finaldayofweek = datatest2.groupby([datatest2.DAYOFWEEK,datatest2.STATION]).sum()

In [22]:
day_of_week = finaldayofweek.reset_index()

In [23]:
whatdayindex = lambda x: daysofweek.index(x)

In [24]:
day_of_week['DAYOFWEEKVALUE'] = day_of_week.DAYOFWEEK.apply(whatdayindex)  


day_of_week_final = day_of_week.sort_values(['DAYOFWEEKVALUE','dTOTAL'], ascending = [True, False])
#data['HOUR'] = data.DATETIME.apply(lambda x: x.hour)

In [25]:
day_of_week_final = day_of_week_final.drop(['ENTRIES', 'EXITS', 'HOUR', 'dENTRIES', 'dEXITS', 'DAYOFWEEKVALUE'],1)

#df = df.drop('column_name', 1)

In [26]:
day_of_week_final.head()

,DAYOFWEEK,STATION,dTOTAL
713,Monday,TIMES SQ-42 ST,3943
377,Monday,125 ST,2314
429,Monday,34 ST-PENN STA,2184
619,Monday,JKSN HT-ROOSVLT,2111
383,Monday,145 ST,1841


day_of_week_final is FINAL dataframe for each day of week